In [1]:
import requests
import time
import pandas as pd
import json
from datetime import datetime
from time import gmtime, strftime

#### Getting basic understanding of what the json format is and does 
``` python
url = 'https://www.reddit.com/r/psychology.json' #mentalhealth #sleep

headers = {'user-agent': 'katychowchow_using_reddit-for_DSI_project'} # tells reddit who you are

res = requests.get(url, headers = headers)

data_json = res.json()

data_json.keys()
data_json['data'].keys()
```

``` python
data_json['data']['before']
data_json['data']['after']
type(data_json['data']['children'])

data_json['data']['children']

data_json['data']['after']

[i['data'] for i in data_json['data']['children'][:2]]

[i['data']['title'] for i in data_json['data']['children']]
```

* this is a dictionary
* you can get everything you need within that
* you can also make it into a dataframe

#### Continuation of exploration of the json format for a test subreddit

```python
test = [i['data']['url'] for i in data_json['data']['children']]

len(test)

# how do we grab posts after the first 27??? 

from pandas.io.json import json_normalize

# this does not work bc data.children.data is a list and you cannot flatten a list...
json_normalize(data_json['data']['children']['data'])

data_json['data']['children']

print(data_json['data']['after'])
print(data_json['data']['children'][0]['data']['title'])
print(data_json['data']['children'][0]['data']['num_comments'])
print(data_json['data']['children'][0]['data']['downs'])
print(data_json['data']['children'][0]['data']['ups'])
print(data_json['data']['children'][0]['data']['likes'])
print(data_json['data']['children'][0]['data']['score'])
print(data_json['data']['children'][0]['data']['view_count'])
print(data_json['data']['children'][0]['data']['subreddit'])
print(data_json['data']['children'][0]['data']['url'])

# num_comments, downs, likes, score, ups, view_count, subreddit
```

#### First go at creating loop to scrape the data 
```python
# key = value pattern after ? query string (dict)
# param = {'after':'t3_a5mdz8'}
headers = {'user-agent': 'katychowchow_using_reddit-for_DSI_project'} 
# requests.get(url, params=param, headers=headers)
posts_psych = []
after = None
for i in range(1):
    print(i)
    if after == None:
        params = {}
    else:
        params = {'after': after}
    url = 'https://www.reddit.com/r/psychology.json'
    res = requests.get(url,params=param,headers=headers)
    if res.status_code == 200:
        data_json = res.json()
#         posts.extend(data_json['data']['children'])
        after = data_json['data']['after']
        for row in data_json['data']['children']:
            post_psych = {}
            post_psych['subreddit'] = row['data']['subreddit']
            post_psych['title'] = row['data']['title']
            post_psych['num_comments'] = row['data']['num_comments']
            post_psych['ups'] = row['data']['ups']
            post_psych['downs'] = row['data']['downs']
            post_psych['likes'] = row['data']['likes']
            post_psych['score'] = row['data']['score']
            post_psych['view_count'] = row['data']['view_count']
            post_psych['url'] = row['data']['url']
            post_psych['timestamp'] = datetime.now()
            posts_psych.append(post_psych)
    else:
        print(res.status_code)
        break
    time.sleep(1)

psych_subreddit_posts = pd.DataFrame(posts_psych)

psych_subreddit_posts.head()
# psych_subreddit_posts.shape
```

#### Loop did not work test 1 
```python
subreddits = ['psychology','mentalhealth','sleep']
after = None
posts = []

for i in subreddits:    
    url = 'https://www.reddit.com/r/'+i+'.json'
    res = requests.get(url,params=param,headers=headers)
    for i in range(2):
        print(i)
        if after == None:
            params = {}
        else:
            params = {'after':after}
        if res.status_code == 200:
            data_json = res.json()
            after = data_json['data']['after']
# #             posts.extend(data_json['data']['children'])
            for row in data_json['data']['children']:
                post = {}
                post['subreddit'] = row['data']['subreddit']
                post['title'] = row['data']['title']
                post['num_comments'] = row['data']['num_comments']
                post['ups'] = row['data']['ups']
                post['downs'] = row['data']['downs']
                post['likes'] = row['data']['likes']
                post['score'] = row['data']['score']
                post['view_count'] = row['data']['view_count']
                post['url'] = row['data']['url']
                posts.append(post)
        else:
            print(res.status_code)
            break
        time.sleep(1.5)         
        ```       

#### another iteration of trying to loop through subreddits as a list then scraping data
```python
subreddits = ['psychology','mentalhealth','sleep']
for i in subreddits:    
    url = 'https://www.reddit.com/r/'+i+'.json'
    res = requests.get(url,params=param,headers=headers)
    print(res.status_code)
```

#### More Trial and Error
```python
subreddits = ['psychology','mentalhealth','sleep']
after = None
posts = []

for i in range(2):
    print(i)
    for i in subreddits:    
        url = 'https://www.reddit.com/r/'+i+'.json'
        res = requests.get(url,params=param,headers=headers)
        if after == None:
            params = {}
        else:
            params = {'after':after}
        if res.status_code == 200:
            data_json = res.json()
            after = data_json['data']['after']
#             posts.extend(data_json['data']['children'])
            for row in data_json['data']['children']:
                post = {}
                post['subreddit'] = row['data']['subreddit']
                post['title'] = row['data']['title']
                post['num_comments'] = row['data']['num_comments']
                post['ups'] = row['data']['ups']
                post['downs'] = row['data']['downs']
                post['likes'] = row['data']['likes']
                post['score'] = row['data']['score']
                post['view_count'] = row['data']['view_count']
                post['url'] = row['data']['url']
                posts.append(post)
        else:
            print(res.status_code)
            break
        time.sleep(1.5)       
        ```

In [16]:
# pd.DataFrame(posts)

,downs,likes,num_comments,score,subreddit,title,ups,url,view_count
0,0,None,2,7,psychology,“An additional reason to abandon learning styl...,7,https://digest.bps.org.uk/2018/12/12/an-additi...,None
1,0,None,2,34,psychology,Internet therapy apps reduce depression sympto...,34,https://news.iu.edu/stories/2018/12/iub/releas...,None
2,0,None,0,2,psychology,The Effect of Memory in Inducing Pleasant Emot...,2,https://www.nature.com/articles/s41598-018-358...,None
3,0,None,1,2,psychology,Ego-depletion gives an incomplete (and pessimi...,2,https://www.psychologytoday.com/ca/blog/ritual...,None
4,0,None,9,360,psychology,Simply telling people they had a gene that low...,360,https://www.sciencemag.org/news/2018/12/just-t...,None
5,0,None,0,0,psychology,Research reveals that the body’s two branches ...,0,https://psychologycompass.com/blog/improve-focus/,None
6,0,None,0,1,psychology,Are you stressed? Your eyes may provide a wind...,1,https://munews.missouri.edu/news-releases/2018...,None
7,0,None,0,0,psychology,How to use IQ to hire for EQ,0,https://www.forbes.com/sites/forbeslacouncil/2...,None
8,0,None,1,7,psychology,"'Hangxiety' higher in shy people - ""Very shy p...",7,https://www.sciencedaily.com/releases/2018/12/...,None
9,0,None,40,1006,psychology,Research shows that building self-restraint ca...,1006,https://psychologycompass.com/blog/self-restra...,None


### Final Loop to scrape data

In [1]:
# first create header for reddit to see that is somewhat meaningful
header = {'user-agent': 'katychowchow_using_reddit-for_DSI_project'}

In [6]:
# final function
def scrape_reddit_posts(url_tag, header, loops, waittime, filename): 
    posts = []
    after = None
    for i in range(loops):
        print(i)
        if after == None:
            params = {}
        else:
            params = {'after': after}
        # url_tag needs to have .json in it
        url = 'https://www.reddit.com/r/'+url_tag
        res = requests.get(url,params=params,headers=header)
        if res.status_code == 200:

            data_json = res.json()
            after = data_json['data']['after']
            for row in data_json['data']['children']:
                post = {}
                post['subreddit'] = row['data']['subreddit']
                post['title'] = row['data']['title']
                post['num_comments'] = row['data']['num_comments']
                post['ups'] = row['data']['ups']
                post['downs'] = row['data']['downs']
                post['likes'] = row['data']['likes']
                post['score'] = row['data']['score']
                post['view_count'] = row['data']['view_count']
                post['url'] = row['data']['url']
                post['time_of_pull'] = datetime.now()
                posts.append(post)
        else:
            print(res.status_code)
            break
        time.sleep(waittime)   
    df = pd.DataFrame(posts)
    now = datetime.now()
    df.to_csv(filename+now.strftime('%Y%m%d%H%S')+'.csv',index=False)
#     return df

In [7]:
scrape_reddit_posts('psychology.json',header,40,1.5,'Reddit_Psych_')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


In [10]:
scrape_reddit_posts('mentalhealth.json',header,40,1.5,'Reddit_MH_')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


In [11]:
scrape_reddit_posts('sleep.json',header,40,1.5,'Reddit_Sleep_')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
